# ニューラルネットワークの学習

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
from PIL import Image
import pickle
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

### 損失関数

**交差エントロビー誤差**

In [2]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))


t = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
y1 = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])
y2 = np.array([0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])

print(f'{cross_entropy_error(y1, t)}')
print(f'{cross_entropy_error(y2, t)}')


0.510825457099338
2.3025840929945454


**ミニバッチ学習**

In [3]:
(X_train, t_train), (X_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
# print(X_train.shape)
# print(t_train.shape)

train_size = X_train.shape[0] # 学習データの個数
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size) # バッチサイズ分データのインデックスをランダムに取り出し
print(batch_mask)
print()

[ 5746  9707  5413 21196 18949 38859 31959 43431 36018 36155]



### 数値微分

In [4]:
def numerical_diff(f, x):
    h = 10e-50
    return (f(x + h) - f(x)) / h



**中心差分**

In [5]:
def numerical_diff(f, x):
    h = 1e-4
    return (f(x + h) - f(x - h)) / (2 * h)

**偏微分**

In [6]:
# 関数の定義
# f(x0, x1) = x0 ** 2 + x1 ** 2
def function_2(x):
    return x[0]**2 + x[1]**2

x0 = 3, x = 4のときのx0に対する偏微分を求めよ

In [7]:
def function_tmp1(x0):
    # print(x0 * x0 + 4.0 ** 2.0)
    return x0 * x0 + 4.0 ** 2.0

x0_grad = numerical_diff(function_tmp1, 3.0)
print(x0_grad)

6.00000000000378


x0 = 3, x = 4のときのx1に対する偏微分を求めよ

In [8]:
def function_tmp2(x1):
    # print(x0 * x0 + 4.0 ** 2.0)
    return 3.0 ** 2.0 + x1 * x1

x0_grad = numerical_diff(function_tmp1, 4.0)
print(x0_grad)

7.999999999999119


**勾配の計算**

In [9]:
def numerical_gradient(f, x):
    """勾配を求める
    
    [Parameter]
    ---------
    f : 最小値を求める関数
    x : x0, x1パラメータの初期値
    
    [Return] 
    ---------
    grad : 勾配
    """
    h = 1e-4 # 0.0001
    grad = np.zeros(x.size) # 勾配を0で初期化
    
    for idx in range(x.size):
        tmp_val = x[idx]
        # print(f'{idx + 1} 個目の重みの勾配を計算 {tmp_val}')
        
        # f(x + h)の計算
        x[idx] = tmp_val + h 
        fxh1 = f(x)
    
        # f(x - h)の計算
        x[idx] = tmp_val - h 
        fxh2 = f(x)
        
        # (f(x + h) - f(x - h)) / (2 * h)の計算
        grad[idx] = (fxh1 - fxh2) / (2 * h)

        x[idx] = tmp_val # 値を元に戻す
        
    # print(grad)
    return grad

numerical_gradient(function_2, np.array([0.3, 0.4]))
numerical_gradient(function_2, np.array([0.0, 2.0]))
numerical_gradient(function_2, np.array([3.0, 0.0]))


array([6., 0.])

**勾配降下法**

In [10]:
# function_2関数の最小値を勾配降下法を使って探す
def function_2(x):
    return x[0]**2 + x[1]**2

init_x = np.array([-3.0, 4.0])

def gradient_descent(f, init_x, lr=0.1, step_num=100):
    """勾配降下法
    
    --------
    [Parameter]
    f       : 最小値を求める関数
    init_x  : パラメータx0,x1の初期値
    lr      : 学習率
    step_num: パラメータを更新する回数
    
    -------
    [Return]
    x       : 更新したパラメータの最終結果
    """
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
        
        if i+1 in [1, 2, 3, 4, 5, 10, 30, 50, 80, 100]:
            print(f'Step{i + 1}回目: 勾配{grad}')
            print(f'Step{i + 1}回目: パラメータ{x}')
        
    return x

gradient_descent(function_2, init_x, lr=0.01, step_num=100)
# gradient_descent(function_2, init_x, lr=10.0, step_num=100) # 学習率が大きすぎる
# gradient_descent(function_2, init_x, lr=1e-10, step_num=100) # 学習率が小さすぎる


Step1回目: 勾配[-6.  8.]
Step1回目: パラメータ[-2.94  3.92]
Step2回目: 勾配[-5.88  7.84]
Step2回目: パラメータ[-2.8812  3.8416]
Step3回目: 勾配[-5.7624  7.6832]
Step3回目: パラメータ[-2.823576  3.764768]
Step4回目: 勾配[-5.647152  7.529536]
Step4回目: パラメータ[-2.76710448  3.68947264]
Step5回目: 勾配[-5.53420896  7.37894528]
Step5回目: パラメータ[-2.71176239  3.61568319]
Step10回目: 勾配[-5.00248657  6.6699821 ]
Step10回目: パラメータ[-2.45121842  3.26829123]
Step30回目: 勾配[-3.33969991  4.45293322]
Step30回目: パラメータ[-1.63645296  2.18193728]
Step50回目: 勾配[-2.22961029  2.97281371]
Step50回目: パラメータ[-1.09250904  1.45667872]
Step80回目: 勾配[-1.21621745  1.62162327]
Step80回目: パラメータ[-0.59594655  0.7945954 ]
Step100回目: 勾配[-0.81195646  1.08260862]
Step100回目: パラメータ[-0.39785867  0.53047822]


array([-0.39785867,  0.53047822])

**ニューラルネットワークに対する勾配**

In [11]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class simpleNet:
    def __init__(self) -> None:
        """重みを初期化(2行3列)
        """
        self.W = np.random.randn(2, 3)
        
    def predict(self, x):
        """入力値を受け取り、各ノードの加重和を計算
        """
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        """損失関数を求める
        
        [Parameter]
        -----------
        x : 入力値
        t : 正解ラベル
        
        [Return]
        -----------
        loss : 損失関数
        """
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        
        return loss


In [12]:
net = simpleNet()
# print(net.W) # 重み
x = np.array([0.6, 0.9]) # 入力値
p = net.predict(x)
# print(p) # 加重和
# print(np.argmax(p)) # 最大値のインデックス

t = np.array([0, 0, 1])
net.loss(x, t)

1.8562116818445138

In [19]:
# f = lambda w: net.loss(x, t)
def f(w):
    
    return net.loss(x, t)

def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 値を元に戻す
        it.iternext()   
        
    return grad

dw = numerical_gradient(f, net.W) # 勾配の計算

[[ 0.03895352  0.46728812 -0.50624164]
 [ 0.05843028  0.70093218 -0.75936247]]

重み [[-0.40257729  0.17859138  0.96420011]
 [-0.36479206  2.00840306 -0.30003535]]
